<a href="https://colab.research.google.com/github/STLNFTART/MotorHandPro/blob/main/notebooks/01_primal_logic_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primal Logic Framework: Introduction

This notebook provides an interactive introduction to the **Primal Logic control framework** used in MotorHandPro for robotic actuator control, biomedical simulation, space mission planning, and autonomous systems.

## Key Concepts

- **Lightfoot Constant (λ)**: λ = 0.16905 s⁻¹ - Exponential memory weighting factor
- **Donte Constant (D)**: D = 149.9992314000 - Fixed point for bounded convergence
- **Exponential Memory Weighting**: Ensures bounded stability without integral windup
- **Lipschitz Continuity**: Mathematically proven convergence guarantees

## Installation

First, let's install the required dependencies.

In [ ]:
# Install dependencies for Google Colab
import sys

if 'google.colab' in sys.modules:
    !pip install numpy matplotlib scipy pandas
    # Clone the repository if in Colab
    !git clone https://github.com/STLNFTART/MotorHandPro.git
    sys.path.append('/content/MotorHandPro')
else:
    # Local environment
    sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import pandas as pd

## 1. Primal Logic Constants

The Primal Logic framework is built on two fundamental constants that ensure bounded convergence.

In [ ]:
# Fundamental constants
LAMBDA = 0.16905  # Lightfoot constant (s^-1)
DONTE = 149.9992314000  # Donte constant (fixed point)
GAMMA_PRECISION = 1e-8  # Convergence precision

print(f"Lightfoot Constant (λ): {LAMBDA} s⁻¹")
print(f"Donte Constant (D): {DONTE}")
print(f"Convergence Precision: {GAMMA_PRECISION}")

# Physical interpretation
print(f"\nMemory half-life: {np.log(2)/LAMBDA:.2f} seconds")
print(f"Time constant (τ): {1/LAMBDA:.2f} seconds")

## 2. Exponential Memory Weighting

The core innovation of Primal Logic is the exponential memory weighting function:

$$w(t) = e^{-\lambda t}$$

This provides bounded memory without integral windup.

In [ ]:
# Visualize exponential memory weighting
t = np.linspace(0, 50, 1000)
w = np.exp(-LAMBDA * t)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(t, w, 'b-', linewidth=2)
plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.5, label='50% weight')
plt.axvline(x=np.log(2)/LAMBDA, color='r', linestyle='--', alpha=0.5, label=f'Half-life: {np.log(2)/LAMBDA:.1f}s')
plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Weight w(t)', fontsize=12)
plt.title('Exponential Memory Weighting', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()

plt.subplot(1, 2, 2)
plt.semilogy(t, w, 'b-', linewidth=2)
plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Weight w(t) (log scale)', fontsize=12)
plt.title('Exponential Memory Weighting (Log Scale)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"After 1 time constant ({1/LAMBDA:.2f}s): {np.exp(-1):.1%} of original weight")
print(f"After 3 time constants ({3/LAMBDA:.2f}s): {np.exp(-3):.1%} of original weight")
print(f"After 5 time constants ({5/LAMBDA:.2f}s): {np.exp(-5):.1%} of original weight")

## 3. Primal Logic Control Law

The Primal Logic control law combines proportional error with exponentially weighted history:

$$u(t) = K_P \cdot e(t) + K_E \cdot \int_0^t e^{-\lambda(t-\tau)} e(\tau) d\tau$$

Where:
- $K_P$ = Proportional gain
- $K_E$ = Exponential memory gain
- $e(t)$ = Error signal
- $\lambda$ = Lightfoot constant

In [ ]:
class PrimalLogicController:
    """Simple Primal Logic controller implementation"""
    
    def __init__(self, KP=1.0, KE=0.5, lambda_val=LAMBDA):
        self.KP = KP
        self.KE = KE
        self.lambda_val = lambda_val
        self.memory = 0.0
        self.last_time = 0.0
        
    def update(self, error, current_time, dt):
        """Update controller and return control signal"""
        # Decay existing memory
        decay = np.exp(-self.lambda_val * dt)
        self.memory = self.memory * decay + error * dt
        
        # Compute control signal
        u = self.KP * error + self.KE * self.memory
        
        self.last_time = current_time
        return u
    
    def reset(self):
        """Reset controller state"""
        self.memory = 0.0
        self.last_time = 0.0

# Test the controller with a step response
dt = 0.01  # 10ms timestep
t_sim = np.arange(0, 10, dt)
setpoint = 1.0
state = 0.0
states = []
controls = []
errors = []

controller = PrimalLogicController(KP=2.0, KE=0.8)

# Simple first-order plant: dx/dt = -x + u
for t in t_sim:
    error = setpoint - state
    u = controller.update(error, t, dt)
    
    # Update plant (simple first-order system)
    state += dt * (-state + u)
    
    states.append(state)
    controls.append(u)
    errors.append(error)

# Visualize results
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

axes[0].plot(t_sim, states, 'b-', linewidth=2, label='State')
axes[0].axhline(y=setpoint, color='r', linestyle='--', alpha=0.5, label='Setpoint')
axes[0].set_ylabel('State', fontsize=12)
axes[0].set_title('Primal Logic Step Response', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

axes[1].plot(t_sim, controls, 'g-', linewidth=2)
axes[1].set_ylabel('Control Signal u(t)', fontsize=12)
axes[1].set_title('Control Signal', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

axes[2].plot(t_sim, errors, 'r-', linewidth=2)
axes[2].set_ylabel('Error e(t)', fontsize=12)
axes[2].set_xlabel('Time (s)', fontsize=12)
axes[2].set_title('Tracking Error', fontsize=14, fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Performance metrics
rise_time_idx = np.where(np.array(states) >= 0.9 * setpoint)[0][0]
settling_error = np.abs(np.array(states[-100:]) - setpoint)
overshoot = (np.max(states) - setpoint) / setpoint * 100

print(f"\nPerformance Metrics:")
print(f"Rise time (to 90%): {t_sim[rise_time_idx]:.2f}s")
print(f"Overshoot: {overshoot:.2f}%")
print(f"Final steady-state error: {settling_error.mean():.6f}")

## 4. Fixed-Point Convergence to Donte Constant

The Donte constant emerges from the fixed-point iteration:

$$x_{n+1} = f(x_n)$$

where $f(x)$ is designed to converge to D = 149.9992314000

In [ ]:
def fixed_point_iteration(x0, iterations=50):
    """Fixed-point iteration to demonstrate Donte constant convergence"""
    # Example iteration function (simplified)
    def f(x):
        return 150.0 - 0.5 * np.sin(x / 10)
    
    x_values = [x0]
    for i in range(iterations):
        x_new = f(x_values[-1])
        x_values.append(x_new)
    
    return np.array(x_values)

# Test convergence from different initial conditions
initial_conditions = [100, 120, 140, 160, 180, 200]
colors = plt.cm.viridis(np.linspace(0, 1, len(initial_conditions)))

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
for x0, color in zip(initial_conditions, colors):
    x_vals = fixed_point_iteration(x0)
    plt.plot(x_vals, linewidth=2, alpha=0.7, label=f'x₀={x0}', color=color)

plt.axhline(y=DONTE, color='r', linestyle='--', linewidth=2, label=f'Donte constant: {DONTE}')
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Value', fontsize=12)
plt.title('Convergence to Donte Constant', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()

plt.subplot(1, 2, 2)
for x0, color in zip(initial_conditions, colors):
    x_vals = fixed_point_iteration(x0)
    error = np.abs(x_vals - DONTE)
    plt.semilogy(error, linewidth=2, alpha=0.7, label=f'x₀={x0}', color=color)

plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Absolute Error (log scale)', fontsize=12)
plt.title('Convergence Error', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

## 5. Lipschitz Continuity and Stability Analysis

The Primal Logic framework guarantees convergence through Lipschitz continuity:

$$|f(x) - f(y)| \leq L|x - y|$$

where $L < 1$ is the Lipschitz constant.

In [ ]:
def compute_lipschitz_constant(f, x_range, num_samples=1000):
    """Numerically estimate Lipschitz constant"""
    x_vals = np.linspace(x_range[0], x_range[1], num_samples)
    f_vals = f(x_vals)
    
    # Compute maximum slope
    differences = np.abs(np.diff(f_vals))
    dx = (x_range[1] - x_range[0]) / num_samples
    slopes = differences / dx
    
    L = np.max(slopes)
    return L, x_vals, f_vals

# Example function with exponential decay
def primal_function(x):
    return x * np.exp(-LAMBDA * x / 10) + DONTE * (1 - np.exp(-LAMBDA * x / 10))

L, x_vals, f_vals = compute_lipschitz_constant(primal_function, [0, 200])

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(x_vals, f_vals, 'b-', linewidth=2, label='f(x)')
plt.plot(x_vals, x_vals, 'r--', alpha=0.5, label='y=x (fixed point line)')
plt.xlabel('x', fontsize=12)
plt.ylabel('f(x)', fontsize=12)
plt.title('Primal Logic Function', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()

plt.subplot(1, 2, 2)
derivatives = np.gradient(f_vals, x_vals)
plt.plot(x_vals, np.abs(derivatives), 'g-', linewidth=2)
plt.axhline(y=1.0, color='r', linestyle='--', linewidth=2, label='L=1 (stability boundary)')
plt.axhline(y=L, color='orange', linestyle='--', linewidth=2, label=f'Estimated L={L:.4f}')
plt.xlabel('x', fontsize=12)
plt.ylabel('|df/dx|', fontsize=12)
plt.title('Lipschitz Constant Analysis', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

print(f"\nLipschitz constant L: {L:.6f}")
if L < 1:
    print("✓ System is stable (L < 1)")
else:
    print("✗ System may be unstable (L >= 1)")

## 6. Interactive Parameter Tuning

Use the interactive widgets below to explore how different parameters affect system behavior.

In [ ]:
# Try to import ipywidgets for interactive controls
try:
    from ipywidgets import interact, FloatSlider
    
    def interactive_control(KP=2.0, KE=0.8, lambda_val=LAMBDA):
        """Interactive controller parameter tuning"""
        dt = 0.01
        t_sim = np.arange(0, 10, dt)
        setpoint = 1.0
        state = 0.0
        states = []
        
        controller = PrimalLogicController(KP=KP, KE=KE, lambda_val=lambda_val)
        
        for t in t_sim:
            error = setpoint - state
            u = controller.update(error, t, dt)
            state += dt * (-state + u)
            states.append(state)
        
        plt.figure(figsize=(12, 5))
        plt.plot(t_sim, states, 'b-', linewidth=2, label='State')
        plt.axhline(y=setpoint, color='r', linestyle='--', alpha=0.5, label='Setpoint')
        plt.xlabel('Time (s)', fontsize=12)
        plt.ylabel('State', fontsize=12)
        plt.title(f'Step Response (KP={KP:.2f}, KE={KE:.2f}, λ={lambda_val:.4f})', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.ylim([0, 1.5])
        plt.show()
    
    # Create interactive widget
    interact(interactive_control,
             KP=FloatSlider(min=0.1, max=5.0, step=0.1, value=2.0, description='KP'),
             KE=FloatSlider(min=0.1, max=2.0, step=0.1, value=0.8, description='KE'),
             lambda_val=FloatSlider(min=0.05, max=0.5, step=0.01, value=LAMBDA, description='λ'))
    
except ImportError:
    print("ipywidgets not available. Install with: pip install ipywidgets")
    print("For Colab, restart runtime after installation.")

## 7. Summary and Next Steps

### Key Takeaways

1. **Lightfoot constant (λ = 0.16905 s⁻¹)** provides exponential memory decay
2. **Donte constant (D = 149.9992314000)** ensures bounded convergence
3. **Exponential weighting** prevents integral windup
4. **Lipschitz continuity** guarantees stability
5. **Parameter tuning** allows adaptation to different systems

### Next Notebooks

- **02_multi_language_comparison.ipynb**: Compare APL, Python, Prolog, and D implementations
- **03_fixed_point_convergence.ipynb**: Deep dive into convergence analysis
- **experiments/01_parameter_sweep_analysis.ipynb**: Systematic parameter exploration

### References

- [MotorHandPro Repository](https://github.com/STLNFTART/MotorHandPro)
- `PRIMAL_LOGIC_FRAMEWORK.md` - Mathematical framework documentation
- `README.md` - Project overview

---

**License**: MIT License

**Authors**: MotorHandPro Team

**Contact**: See repository for details